In [ ]:
import numpy as np
import cv2
from PIL import Image, ImageOps, ImageDraw, ImageFont
import csv

# Constant for dialogue font size (in points) and multiline spacing
DIALOGUE_FONT_SIZE = 60
TEXT_SPACING = 4

# -------------------------------------------------
# HELPER FUNCTION: CHOOSE FONT BASED ON LANGUAGE
# -------------------------------------------------
def get_font_for_text(text, font_size, language):
    """
    Returns a TrueType font object.
    For English dialogues, uses Arial; for Kannada, uses NotoSansKannada-Regular.
    Adjust font paths as needed.
    """
    if language.lower() == "english":
        try:
            return ImageFont.truetype("/System/Library/Fonts/Supplemental/Arial.ttf", font_size)
        except Exception:
            return ImageFont.load_default()
    else:
        try:
            return ImageFont.truetype("/Library/Fonts/NotoSansKannada-Regular.ttf", font_size)
        except Exception:
            return ImageFont.load_default()

# -------------------------------------------------
# HELPER FUNCTION: CARTOONIZE IMAGE
# -------------------------------------------------
def cartoonize(image, blur_value=7, edge_block_size=43, edge_c=7):
    """
    Applies a cartoon effect to the given image.
    """
    img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, blur_value)
    edges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                  cv2.THRESH_BINARY, edge_block_size, edge_c)
    cartoon = cv2.bitwise_and(img_cv, img_cv, mask=edges)
    return Image.fromarray(cv2.cvtColor(cartoon, cv2.COLOR_BGR2RGB))

# -------------------------------------------------
# WRAP TEXT FUNCTION (preserving newlines)
# -------------------------------------------------
def wrap_text(text, font, max_width, draw):
    """
    Wraps text into multiple lines so that any newline characters in the original text are preserved.
    Each paragraph is wrapped individually.
    """
    paragraphs = text.split("\n")
    wrapped_paragraphs = []
    for para in paragraphs:
        words = para.split()
        if not words:
            wrapped_paragraphs.append("")
            continue
        line = words[0]
        lines = []
        for word in words[1:]:
            candidate_line = line + " " + word
            bbox = draw.textbbox((0, 0), candidate_line, font=font)
            if (bbox[2] - bbox[0]) <= max_width:
                line = candidate_line
            else:
                lines.append(line)
                line = word
        lines.append(line)
        wrapped_paragraphs.append("\n".join(lines))
    return "\n".join(wrapped_paragraphs)

# -------------------------------------------------
# FUNCTION: PROCESS SCRIPT PAGE (ONE PDF PAGE)
# -------------------------------------------------
def process_script_page(language, script_file, page_title, signature, page_number, group_indices, dialogue_offset=0):
    """
    Processes the images and dialogues to produce one PDF page.
    Only the image groups specified by group_indices are used.
    
    For each group (strip), images are resized to a common height (preserving aspect ratios)
    and horizontally stacked. The CSV is read and dialogues for this page are taken starting
    from dialogue_offset.
    
    Each image’s dialogue is drawn inside its inner area (i.e. within the border)
    so that its bottom edge exactly aligns with (base.height - border_size).
    
    Annotations (title, page number, signature) are added.
    """
    # 1) READ DIALOGUES FROM CSV
    dialogues = []
    with open(script_file, newline='', encoding="utf-8") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row:
                dialogues.append(row[0])
    print(f"[DEBUG] Read {len(dialogues)} dialogues from {script_file}.")
    
    # 2) DEFINE ALL IMAGE GROUPS and select groups for this page
    all_groups = [
        ['data/img1.0.jpeg', 'data/img1.1.jpeg', 'data/img1.2.jpeg', 'data/img1.3.jpeg'],
        ['data/img2.0.jpeg', 'data/img2.1.jpeg', 'data/img2.2.jpeg', 'data/img2.3.jpeg'],
        ['data/img3.0.jpeg', 'data/img3.1.jpeg', 'data/img3.2.jpeg', 'data/img3.3.jpeg'],
        ['data/img4.0.jpeg', 'data/img4.1.jpeg', 'data/img4.2.jpeg', 'data/img4.3.jpeg'],
        ['data/img5.0.jpg', 'data/img5.1.jpg', 'data/img5.2.jpg', 'data/img6.0.jpg'],
        ['data/img6.1.jpg', 'data/img6.2.jpg', 'data/img6.3.jpg', 'data/img6.4.jpg'],
        ['data/img7.0.jpg', 'data/img7.1.jpg'],
        ['data/img8.0.jpg', 'data/img8.1.jpg', 'data/img8.2.jpg'],
        ['data/img16.0.jpg', 'data/img16.1.jpg', 'data/img16.2.jpg'],
        ['data/img17.0.jpg', 'data/img17.1.jpg', 'data/img17.2.jpg'],
        ['data/img18.0.jpg', 'data/img18.1.jpg', 'data/img18.2.jpg'],
        ['data/img19.0.jpg', 'data/img19.1.jpg'],
        ['data/img12.0.jpg', 'data/img12.1.jpg', 'data/img12.2.jpg'],
        ['data/img13.0.jpg', 'data/img13.1.jpg', 'data/img13.2.jpg'],
        ['data/img14.0.jpg', 'data/img14.1.jpg'],
        ['data/img15.0.jpg', 'data/img15.1.jpg']
    ]
    groups = [all_groups[i] for i in group_indices]
    expected_dialogues = sum(len(g) for g in groups)
    print(f"[DEBUG] Using {len(groups)} groups; expecting {expected_dialogues} dialogues.")
    if expected_dialogues + dialogue_offset > len(dialogues):
        raise ValueError(f"Not enough dialogues: expected {expected_dialogues + dialogue_offset}, got {len(dialogues)}.")
    dialogues = dialogues[dialogue_offset: dialogue_offset + expected_dialogues]
    
    # Settings for borders and text shadow
    border_size = 50
    border_color = "white"
    TINT_COLOR = (0, 0, 0)
    TRANSPARENCY = 0.4
    OPACITY = int(255 * TRANSPARENCY)
    
    # 3) First pass: Precompute wrapped texts for each image
    wrapped_texts = []
    dialogue_index_local = 0
    for group in groups:
        imgs = [Image.open(path).convert("RGB") for path in group]
        common_height = min(img.height for img in imgs)
        imgs_resized = [img.resize((int(img.width * common_height / img.height), common_height), Image.Resampling.LANCZOS) for img in imgs]
        imgs_cartoon = [cartoonize(img) for img in imgs_resized]
        imgs_bordered = [ImageOps.expand(img, border=border_size, fill=border_color) for img in imgs_cartoon]
        for img in imgs_bordered:
            dummy_img = img.copy().convert("RGBA")
            dummy_draw = ImageDraw.Draw(dummy_img)
            text = dialogues[dialogue_index_local]
            font = get_font_for_text(text, DIALOGUE_FONT_SIZE, language)
            max_text_width = dummy_img.width - 2 * border_size
            wrapped = wrap_text(text, font, max_text_width, dummy_draw)
            wrapped_texts.append((wrapped, font))
            dialogue_index_local += 1
    
    # 4) Second pass: Draw dialogues on each image and combine each strip horizontally
    final_strips = []
    dialogue_index_local = 0
    for group in groups:
        imgs = [Image.open(path).convert("RGB") for path in group]
        common_height = min(img.height for img in imgs)
        imgs_resized = [img.resize((int(img.width * common_height / img.height), common_height), Image.Resampling.LANCZOS) for img in imgs]
        imgs_cartoon = [cartoonize(img) for img in imgs_resized]
        imgs_bordered = [ImageOps.expand(img, border=border_size, fill=border_color) for img in imgs_cartoon]
        
        strip_imgs_with_text = []
        for img in imgs_bordered:
            base = img.convert("RGBA")
            overlay = Image.new("RGBA", base.size, (0,0,0,0))
            draw_overlay = ImageDraw.Draw(overlay)
            wrapped, font = wrapped_texts[dialogue_index_local]
            # Use multiline_textbbox for accurate measurement (with fixed spacing)
            bbox = draw_overlay.multiline_textbbox((0, 0), wrapped, font=font, spacing=TEXT_SPACING)
            actual_text_height = bbox[3] - bbox[1]
            # The inner area is defined from y = border_size to y = base.height - border_size.
            # Set text_y so that the text's bottom aligns exactly with (base.height - border_size)
            text_y = (base.height - border_size) - actual_text_height
            if text_y < border_size:
                text_y = border_size
            # Draw background rectangle strictly within the inner area
            draw_overlay.rectangle((border_size, text_y, base.width - border_size, base.height - border_size),
                                   fill=TINT_COLOR + (OPACITY,))
            draw_overlay.multiline_text((border_size, text_y), wrapped, font=font, fill=(255, 255, 255), spacing=TEXT_SPACING)
            combined = Image.alpha_composite(base, overlay).convert("RGB")
            strip_imgs_with_text.append(combined)
            dialogue_index_local += 1
        # Horizontally stack images in this strip
        strip_array = np.hstack([np.array(im) for im in strip_imgs_with_text])
        strip_img = Image.fromarray(strip_array)
        final_strips.append(strip_img)
    
    # 5) Adjust each strip so that all strips have the same width by scaling up if needed
    target_width = max(strip.width for strip in final_strips)
    scaled_strips = []
    for strip in final_strips:
        if strip.width < target_width:
            scale_factor = target_width / strip.width
            new_height = int(strip.height * scale_factor)
            scaled_strip = strip.resize((target_width, new_height), Image.Resampling.LANCZOS)
            scaled_strips.append(scaled_strip)
        else:
            scaled_strips.append(strip)
    
    # 6) Combine the strips vertically to form one page
    margin_top = 150
    margin_bottom = 100
    line_thickness = 5
    num_strips = len(scaled_strips)
    extra_line_height = (num_strips - 1) * line_thickness
    combined_total_height = sum(strip.height for strip in scaled_strips) + margin_top + margin_bottom + extra_line_height
    combined_width = target_width
    combined_image = Image.new("RGB", (combined_width, combined_total_height), "white")
    draw_combined = ImageDraw.Draw(combined_image)
    y_offset = margin_top
    for i, strip in enumerate(scaled_strips):
        combined_image.paste(strip, (0, y_offset))
        y_offset += strip.height
        if i < num_strips - 1:
            draw_combined.rectangle([(0, y_offset), (combined_width, y_offset + line_thickness)], fill="black")
            y_offset += line_thickness

    # 7) Annotations: Page number, signature, and optional title
    page_number_text = page_number
    title_font = get_font_for_text(page_title if page_title is not None else "", 60, language)
    other_font = get_font_for_text(page_number_text, 40, language)
    
    if page_title is not None:
        bbox_title = draw_combined.textbbox((0, 0), page_title, font=title_font)
        title_text_width = bbox_title[2] - bbox_title[0]
        title_text_height = bbox_title[3] - bbox_title[1]
        title_x = (combined_width - title_text_width) // 2
        title_y = (margin_top - title_text_height) // 2
        draw_combined.text((title_x, title_y), page_title, fill="black", font=title_font)
    
    bbox_page = draw_combined.textbbox((0, 0), page_number_text, font=other_font)
    page_text_width = bbox_page[2] - bbox_page[0]
    draw_combined.text((combined_width - page_text_width - 20, 20),
                       page_number_text, fill="black", font=other_font)
    
    bbox_sig = draw_combined.textbbox((0, 0), signature, font=other_font)
    sig_text_width = bbox_sig[2] - bbox_sig[0]
    sig_text_height = bbox_sig[3] - bbox_sig[1]
    draw_combined.text((combined_width - sig_text_width - 20, combined_total_height - sig_text_height - 20),
                       signature, fill="black", font=other_font)
    
    print(f"[DEBUG] Processed page '{page_number}' successfully.")
    return combined_image

# -------------------------------------------------
# CREATE MULTIPAGE PDF FOR 4 PAGES PER LANGUAGE
# -------------------------------------------------
try:
    # For English, dialogue counts based on groups:
    # Groups [0,1,2,3] have 4+4+4+4 = 16 images; groups [4,5,6,7] have 3+5+2+3 = 13 images.
    # So offsets: Page 1: offset=0, Page 2: offset=16, Page 3: offset=29, Page 4: offset=45.
    page1_eng = process_script_page("english", "data/script-english.csv",
                                "Roz-42", "Created by Puja Barathi Nagarajan", "Page 1", [0,1,2,3], dialogue_offset=0)
    page2_eng = process_script_page("english", "data/script-english.csv",
                                None, "Created by Tejas Venkatesh", "Page 2", [4,5,6,7], dialogue_offset=16)
    page3_eng = process_script_page("english", "data/script-english.csv",
                                None, "Created by Manikhandan Mavureddy Senthilvelu", "Page 4", [8,9,10,11], dialogue_offset=29)
    page4_eng = process_script_page("english", "data/script-english.csv",
                                None, "Created by Amit Anveri", "Page 4", [12,13,14,15], dialogue_offset=40)
    
    eng_pages = [page1_eng, page2_eng, page3_eng, page4_eng]
    # Force all English pages to the same width (take the minimum width among them)
    target_width_eng = min(page.width for page in eng_pages)
    eng_pages_resized = [page.resize((target_width_eng, int(page.height * target_width_eng / page.width)), Image.Resampling.LANCZOS) for page in eng_pages]
    
    eng_pages_resized[0].save("comic/storybook-english.pdf", "PDF", resolution=100.0,
                               save_all=True, append_images=eng_pages_resized[1:])
    print("✅ Multipage PDF 'storybook-english.pdf' saved successfully!")
    
    # For Kannada, similar offsets and groups; adjust annotations as needed.
    page1_kn = process_script_page("kannada", "data/script-kannada.csv",
                                "ರೋಜ್ 42", "ಪರಿಚಯ: ಪುಜಾ ಬಾರಥಿ ನಗರಾಜನ್", "ಪುಟ 1", [0,1,2,3], dialogue_offset=0)
    page2_kn = process_script_page("kannada", "data/script-kannada.csv",
                                None, "ತೇಜಸ್ ವೆಂಕಟೇಶ್ ರಚಿಸಿದ್ದಾರೆ", "ಪುಟ 2", [4,5,6,7], dialogue_offset=16)
    page3_kn = process_script_page("kannada", "data/script-kannada.csv",
                                None, "ಮಣಿಖಂಡನ್ ಮಾವುರೆಡ್ಡಿ ಸೆಂಥಿಲ್ವೇಲು ರಚಿಸಿದ್ದಾರೆ", "ಪುಟ 4", [8,9,10,11], dialogue_offset=29)
    page4_kn = process_script_page("kannada", "data/script-kannada.csv",
                                None, "ಅಮಿತ್ ಅನ್ವೇರಿ ರಚಿಸಿದ್ದಾರೆ", "ಪುಟ 4", [12,13,14,15], dialogue_offset=40)
    
    kn_pages = [page1_kn, page2_kn, page3_kn, page4_kn]
    target_width_kn = min(page.width for page in kn_pages)
    kn_pages_resized = [page.resize((target_width_kn, int(page.height * target_width_kn / page.width)), Image.Resampling.LANCZOS) for page in kn_pages]
    
    kn_pages_resized[0].save("comic/storybook-kannada.pdf", "PDF", resolution=100.0,
                              save_all=True, append_images=kn_pages_resized[1:])
    print("✅ Multipage PDF 'storybook-kannada.pdf' saved successfully!")
    
except Exception as e:
    print("Error occurred:", e)


[DEBUG] Read 50 dialogues from data/script-english.csv.
[DEBUG] Using 4 groups; expecting 16 dialogues.
[DEBUG] Processed page 'Page 1' successfully.
[DEBUG] Read 50 dialogues from data/script-english.csv.
[DEBUG] Using 4 groups; expecting 13 dialogues.
[DEBUG] Processed page 'Page 2' successfully.
[DEBUG] Read 50 dialogues from data/script-english.csv.
[DEBUG] Using 4 groups; expecting 11 dialogues.
[DEBUG] Processed page 'Page 4' successfully.
[DEBUG] Read 50 dialogues from data/script-english.csv.
[DEBUG] Using 4 groups; expecting 10 dialogues.
[DEBUG] Processed page 'Page 4' successfully.
✅ Multipage PDF 'storybook-english.pdf' saved successfully!
[DEBUG] Read 50 dialogues from data/script-kannada.csv.
[DEBUG] Using 4 groups; expecting 16 dialogues.
